In [92]:
import numpy as np
from dipy.io.streamline import load_trk
from dipy.viz import window, actor
import math
from nibabel import trackvis as tv
from dipy.tracking.streamline import Streamlines
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_data
from dipy.segment.metric import mdf

In [2]:
def show_streamlines(streamlines): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    window.show(ren)

In [3]:
def show_multiple_streamlines(streamline1,streamline2): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamline1))
    ren.add(actor.line(streamline2,colors=(0,1,1)))
    window.show(ren)

 load brain (tractogram)

In [149]:
brain, b_hr = load_trk("brain.trk")

load Arcuate Fasciculus left fiber bundle (AF_L)

In [5]:
bundle, bn_hr = load_trk("AF_L.trk")

In [146]:
cst, bn_hr = load_trk("CST_recognized.trk")

let's visualize them

In [7]:
# first brain

#show_streamlines(brain)

In [8]:
# now bundle

show_streamlines(bundle[:50])

/Users/rp7/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [9]:
(bundle[0].shape)
len(bundle[0])

331

In [147]:
# get labels

def getLabels(brain, bundle):
    labels = np.zeros(len(brain))
    npbrain = np.array(brain)
    for i in range(len(bundle)):
        for j in range(len(brain)):
            if((npbrain[j].shape[0]==bundle[i].shape[0])):
    #         print(i,j)
                if np.allclose(npbrain[j], bundle[i]):
                    labels[j] = 1
                    break
        if(i%100==0):
            print(i,end=" ")
    return labels

In [ ]:
l = getLabels(brain, bundle)
labels[l==l] = 1
l = getLabels(brain, cst)
labels[l==l] = 2

In [10]:
#Find Frechet distance
# From : https://gist.github.com/MaxBareiss/ba2f9441d9455b56fbc9

# Euclidean distance.
def euc_dist(pt1,pt2):
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0])+(pt2[1]-pt1[1])*(pt2[1]-pt1[1]))

def _c(ca,i,j,P,Q):
    if ca[i,j] > -1:
        return ca[i,j]
    elif i == 0 and j == 0:
        ca[i,j] = euc_dist(P[0],Q[0])
    elif i > 0 and j == 0:
        ca[i,j] = max(_c(ca,i-1,0,P,Q),euc_dist(P[i],Q[0]))
    elif i == 0 and j > 0:
        ca[i,j] = max(_c(ca,0,j-1,P,Q),euc_dist(P[0],Q[j]))
    elif i > 0 and j > 0:
        ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),euc_dist(P[i],Q[j]))
    else:
        ca[i,j] = float("inf")
    return ca[i,j]

""" Computes the discrete frechet distance between two polygonal lines
Algorithm: http://www.kr.tuwien.ac.at/staff/eiter/et-archive/cdtr9464.pdf
P and Q are arrays of 2-element arrays (points)
"""
def frechetDist(P,Q):
    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca,len(P)-1,len(Q)-1,P,Q)

In [11]:
# Fast Dynamic time warping
# From: https://github.com/slaypni/fastdtw

import numpy as np
from fastdtw import fastdtw

In [66]:
from dipy.align.streamlinear import BundleMinDistanceMetric
from dipy.align.streamlinear import set_number_of_points
# BundleMinDistanceMetric().distance(cBrain3D[0],cBrain3D[1])

def bmd(a,b):
    static = set_number_of_points(a, len(a))
    moving = set_number_of_points(b, len(b))
    BMD = BundleMinDistanceMetric()
    BMD.setup(static, moving)
    x0 = np.array([0, 0, 0, 0, 0, 0, 1., 1., 1, 0, 0, 0])  # affine
    bmd_value = BMD.distance(x0.tolist())
    return bmd_value

In [101]:
bmd(brain3D[0],brain3D[7])
# brain3D[1].shape

5.380595098434914

In [111]:
# Get distance between 2 curves
from scipy.spatial.distance import euclidean

def distanceBetweenCurves(c1, c2, algo='fastdtw'):
    if algo=='fastdtw':
        distance, path = fastdtw(c1,c2, radius = 1, dist = euclidean) # check out the 'radius' parameter to tune the distance
        return distance
    elif algo=='frechet':
        return frechetDist(c1,c2)
    elif algo=='bmd':
        return bmd(c1,c2)
    elif algo=='mdf':
        return mdf(c1,c2)

In [14]:
from sklearn.decomposition import PCA
def project2D(streamline):
    pca = PCA(n_components=2)
    newBundle = pca.fit_transform(streamline)
    newStreamline = Streamlines([(np.hstack((newBundle,np.zeros((newBundle.shape[0])).reshape((-1,1)))))])
    return newStreamline, pca.explained_variance_ratio_, pca.singular_values_

In [15]:
# print(pca.explained_variance_ratio_) 
# print(pca.singular_values_)

In [16]:
# st = Streamlines([newBundle])
def convertToAngles(streamline2D):
    streamline2D = np.array(streamline2D)
#     print((streamline2D))
    angles = np.zeros(streamline2D.shape[0])
    angles[0] = 1
#     print(streamline2D.shape)
    # handle the direction of start
#     print(streamline2D.shape)
    for point in range(1,streamline2D.shape[0]):
#         print(point)
        diff = streamline2D[point] - streamline2D[point-1]
#         print(streamline2D[point], streamline2D[point-1])
        if(diff[0]==0):
            angles[point] = angles[point-1]
        else:
            angles[point] = math.degrees(math.atan(diff[1]/diff[0]))
    return angles


In [17]:
def getDataLD(bundle):
    data2D = []
    angularData = []
    for index in range(len(bundle)):
        newStreamline, varRatio, singVal = project2D(np.array(bundle[index]))
        data2D.append(newStreamline[0][:,:2])
#         print(np.array(bundle[index]))
        # append these values to store them 
#         print(len(newStreamline))
        streamlineAsAngle = convertToAngles(newStreamline[0])
#         print(streamlineAsAngle)
        angularData.append(streamlineAsAngle)
    
    return angularData, data2D

In [18]:
bundle1D, bundle2D = getDataLD(bundle[1950:])
len(bundle1D)
brain1D, brain2D = getDataLD(brain[1950:2150])
brain3D = np.array(brain[1950:2150]).tolist()
# show_multiple_streamlines(newStreamline,bundle[0:1])

In [19]:
# set labels
labels = np.ones(len(brain1D))
labels[50:]*=0

In [20]:
# center the data
def centerData(data):
    for i in range(len(data)):
        mean = np.mean(data[i],axis=0)
        data[i] = data[i]-mean
    return data

cBrain1D = centerData(brain1D)
cBrain2D = centerData(brain2D)
cBrain3D = centerData(brain3D)

In [112]:
distanceBetweenCurves(cBrain3D[3], cBrain3D[40], 'mdf')

11.766499224626674

In [118]:
# create covariance matrix

cov = np.zeros((len(cBrain3D), len(cBrain3D)))
for i in range(len(cBrain3D)):
    for j in range(len(cBrain3D)):
        cov[i][j] = distanceBetweenCurves(cBrain3D[i], cBrain3D[j], 'frechet')
    print(i, end=" ")

KeyboardInterrupt: 

In [114]:
# store the covariance matrix variable to disk
import pickle

# Save
with open('covarianceMDF3D.pkl', 'wb') as f:
    pickle.dump(cov, f)

In [31]:
# Retrieve
cov = []
with open('covariance3D.pkl', 'rb') as f:
    cov = pickle.load(f)

In [117]:
cov.shape

(200, 200)

In [116]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.imshow(cov)

In [144]:
# get centroids for the bundles

streamlines = brain3D
def getCentroids(streamlines, noOfCentroids):
    thresh=1
    qb = QuickBundles(threshold=thresh)
    clusters = qb.cluster(streamlines)
    length = len(clusters.centroids)
    prev = 0
    flag=False
    while(length!=noOfCentroids):
        if(length>noOfCentroids):
            if flag:
                temp = thresh
                thresh= thresh + (thresh-prev)/2.0
                prev = temp
            else:
                prev=thresh
                thresh *= 2.0
        else:
            temp = thresh
            thresh = prev+(thresh-prev)/2.0
            flag = True
        qb = QuickBundles(threshold=thresh)
        clusters = qb.cluster(streamlines)
        length = len(clusters)
    return clusters.centroids

In [145]:
getCentroids(cBrain3D, 3)

[array([[  3.5255    ,  39.49031   ,  -7.988553  ],
        [ -1.3283533 ,  36.69522   ,  -5.962709  ],
        [ -4.1172867 ,  31.38615   ,  -1.431215  ],
        [ -4.3687663 ,  22.310734  ,   3.0781322 ],
        [ -3.7871964 ,  12.1645155 ,   5.7893057 ],
        [ -2.6798606 ,   1.6687901 ,   8.1687355 ],
        [ -2.3622484 ,  -9.075774  ,   9.53447   ],
        [ -0.99746406, -19.206038  ,   7.89226   ],
        [  1.2021804 , -26.300194  ,   2.1432986 ],
        [  3.4617891 , -28.449865  ,  -5.7119584 ],
        [  7.464814  , -27.203264  , -12.042078  ],
        [ 12.833145  , -27.642853  , -14.91183   ]], dtype=float32),
 array([[ -4.526729  ,  39.15395   ,  10.483583  ],
        [  2.2170033 ,  33.915253  ,   9.336281  ],
        [  5.329919  ,  25.052439  ,  10.196454  ],
        [  5.374535  ,  13.649774  ,  12.57605   ],
        [  5.4918966 ,   1.6210372 ,  13.941188  ],
        [  5.2787304 , -10.088643  ,  13.095058  ],
        [  4.035641  , -19.462532  ,   7.980045

In [ ]:
#show_streamlines(cst)

In [ ]:
#show_multiple_streamlines(bundle,cst)

let's see its data type

In [ ]:
#bundle # whole bundle, made of several streamlines

In [ ]:
# ((brain)[0]).shape# number of streamlines in bundle

In [ ]:
# bundle[1] #one streamline in bundle
# (np.array(bundle)[0]).shape
# np.array(bundle)

In [ ]:
# (bundle[1].shape) #one point on one streamline

In [117]:
npbrain = np.array(brain)

In [ ]:
# npbrain[0].shape

In [ ]:
# len(bundle)

these are x, y and z co-ordinates of the point

In [118]:
# get labels

def getLabels(brain, bundle):
    labels = np.zeros(len(brain))
    npbrain = np.array(brain)
    for i in range(len(bundle)):
        for j in range(len(brain)):
            if((npbrain[j].shape[0]==bundle[i].shape[0])):
    #         print(i,j)
                if np.allclose(npbrain[j], bundle[i]):
                    labels[j] = 1
                    break
        if(i%100==0):
            print(i,end=" ")
    return labels

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [21]:
# npbrain[0].shape


In [23]:
n_pts = [len(streamline) for streamline in bundle]

In [ ]:
from dipy.tracking.streamline import (set_number_of_points, nbytes,select_random_set_of_streamlines)
newStreamlines = set_number_of_points(bundle,101)
n_pts_ds = [len(streamline) for streamline in newStreamlines]

In [ ]:
newBrain = set_number_of_points(brain,101)

In [127]:
# newBrain[0].shape
trainset[0].shape

(619,)

In [119]:
# len(npbrain)
npbrain = np.array(brain1D)

In [120]:
import tensorflow as tf
import keras

In [121]:
# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.02
training_steps = 70
batch_size = 100
display_step = 200

# Network Parameters
seq_max_len = 1000 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

trainset = npbrain[1950:2050]
testset = labels[1950:2050]

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 3])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [122]:
def dynamicRNN(x, seqlen, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

In [123]:
tf_session = keras.backend.get_session()
tf_graph = tf.get_default_graph()
# tf.reset_default_graph()
print([n.name for n in tf.get_default_graph().as_graph_def().node])
# tf.contrib.rnn.reset_states()
# with tf_session.as_default():
#      with tf_graph.as_default():
#             tf.reset_default_graph() 
pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

['Placeholder', 'Placeholder_1', 'Placeholder_2', 'random_normal/shape', 'random_normal/mean', 'random_normal/stddev', 'random_normal/RandomStandardNormal', 'random_normal/mul', 'random_normal', 'Variable', 'Variable/Assign', 'Variable/read', 'random_normal_1/shape', 'random_normal_1/mean', 'random_normal_1/stddev', 'random_normal_1/RandomStandardNormal', 'random_normal_1/mul', 'random_normal_1', 'Variable_1', 'Variable_1/Assign', 'Variable_1/read', 'IsVariableInitialized', 'IsVariableInitialized_1', 'init']
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/Users/rp7/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [124]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
#         batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        rangelow = ((step-1)*batch_size)%(trainset.shape[0])
        rangehigh = ((step)*batch_size)%(trainset.shape[0])
        if(rangehigh==rangelow):
            rangehigh=rangehigh-1
        if(rangehigh<rangelow):
            batch_x = np.append(trainset[rangelow:], trainset[:rangehigh])
            batch_y = np.append(testset[rangelow:], testset[:rangehigh])
        else:
            batch_x = trainset[rangelow:rangehigh]
            batch_y = testset[rangelow:rangehigh]
        
        batch_seqlen = []
        for sample in trainset:
            batch_seqlen.append(sample.shape[0])

#         batch_seqlen = batch_seqlen.tolist()
        print(type(batch_seqlen))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

<class 'list'>


ValueError: setting an array element with a sequence.

In [ ]:
class ToySequenceData(object):
    """ Generate sequence of data with dynamic length.
    This class generate samples for training:
    - Class 0: linear sequences (i.e. [0, 1, 2, 3,...])
    - Class 1: random sequences (i.e. [1, 3, 10, 7,...])

    NOTICE:
    We have to pad each sequence to reach 'max_seq_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform thanks to
    'seqlen' attribute that records every actual sequence length.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Random sequence length
            len = np.random.randint(min_seq_len, max_seq_len)
            # Monitor sequence length for TensorFlow dynamic calculation
            self.seqlen.append(len)
            # Add a random or linear int sequence (50% prob)
            if np.random.random() < .5:
                # Generate a linear sequence
                rand_start = np.random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generate a random sequence
                s = [[float(np.random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        print(type(batch_seqlen[0]))
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [ ]:
tr = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
te = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)

In [ ]:
b_x, b_y, b_seqlen = tr.next(100)

In [ ]:
print((b_x))
# print(batch_seqlen)